# Download the data, combine with background and binning example (4C+21.35, flare)

**This tutotrial walks through all the steps needed in creating the combined data set (source + background), and binning the data for the blazar (4C+21.35, flare).**

**For the background we use just the albedo photons.**

**To run this, you need the following files (available on wasabi)**

- COSI-SMEX/DC3/Data/Sources/4C21p35_flare_3months_unbinned_data_filtered_with_SAAcut.fits.gz 
- COSI-SMEX/DC3/Data/Backgrounds/Ge/AlbedoPhotons_3months_unbinned_data_filtered_with_SAAcut.fits.gz  
 

In [1]:
from cosipy import COSILike, BinnedData
from cosipy.spacecraftfile import SpacecraftFile
from cosipy.response.FullDetectorResponse import FullDetectorResponse
from cosipy.util import fetch_wasabi_file

from scoords import SpacecraftFrame

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.stats import poisson_conf_interval

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from threeML import Band, PointSource, Model, JointLikelihood, DataList
from cosipy import Band_Eflux
from astromodels import Parameter

from pathlib import Path

import os
import corner
from threeML import*


08:01:28 WARNING   The naima package is not available. Models that depend on it will not be         ]8;id=782477;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/astromodels/functions/functions_1D/functions.py\functions.py]8;;\:]8;id=299332;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/astromodels/functions/functions_1D/functions.py#47\47]8;;\
                  available                                                                                        

         WARNING   The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it  ]8;id=82924;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/astromodels/functions/functions_1D/functions.py\functions.py]8;;\:]8;id=83186;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/astromodels/functions/functions_1D/functions.py#68\68]8;;\
                  will not be available.                                                                           

         WARNING   The ebltable package is not available. Models that depend on it will not be     ]8;id=774068;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/astromodels/functions/functions_1D/absorption.py\absorption.py]8;;\:]8;id=91408;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/astromodels/functions/functions_1D/absorption.py#33\33]8;;\
                  available                                                                                        

         INFO      Starting 3ML!                                                                     ]8;id=19627;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=934328;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py#39\39]8;;\

         WARNING   WARNINGs here are NOT errors                                                      ]8;id=940848;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=208413;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py#40\40]8;;\

...

         WARNING   Could not import plugin HAWCLike.py. Do you have the relative instrument         ]8;id=70001;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=159262;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py#144\144]8;;\
                  software installed and configured?                                                               

         WARNING   No fermitools installed                                              ]8;id=784966;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/utils/data_builders/fermi/lat_transient_builder.py\lat_transient_builder.py]8;;\:]8;id=393553;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/utils/data_builders/fermi/lat_transient_builder.py#44\44]8;;\

         WARNING   Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal         ]8;id=282445;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=340190;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py#387\387]8;;\
                  performances in 3ML                                                                              

         WARNING   Env. variable MKL_NUM_THREADS is not set. Please set it to 1 for optimal         ]8;id=522652;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=206266;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py#387\387]8;;\
                  performances in 3ML                                                                              

         WARNING   Env. variable NUMEXPR_NUM_THREADS is not set. Please set it to 1 for optimal     ]8;id=916840;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=750505;file:///home/srinadb/miniconda3/envs/cosipy_developer/lib/python3.10/site-packages/threeML/__init__.py#387\387]8;;\
                  performances in 3ML                                                                              

## Get the data

Define the path to the directory to save the data. The data can be downloaded by running the cells below. Each respective cell also gives the wasabi file path and file size.

In [2]:
data_path = Path("/home/srinadb/cosipy/docs/tutorials/spectral_fits/dataset")


Download the source file (9.7 MB)

In [3]:
# 4C+21.35 source
# File size: 9.7 MB

fetch_wasabi_file('COSI-SMEX/DC3/Data/Sources/4C21p35_flare_3months_unbinned_data_filtered_with_SAAcut.fits.gz', output=str(data_path / '4C21p35_flare_3months_unbinned_data_filtered_with_SAAcut.fits.gz'))


{
    "AcceptRanges": "bytes",
    "LastModified": "Thu, 13 Feb 2025 17:00:49 GMT",
    "ContentLength": 18935412,
    "ETag": "\"3ddeed0ad1e01accc413fa5cffcfc13e\"",
    "ContentType": "application/gzip",
    "Metadata": {}
}


Download the background file (2.7 GB)

In [4]:
# albedo photons
# File size: 2.7 GB
# fetch_wasabi_file('COSI-SMEX/DC3/Data/Backgrounds/Ge/AlbedoPhotons_3months_unbinned_data_filtered_with_SAAcut.fits.gz', output=str(data_path / 'AlbedoPhotons_3months_unbinned_data_filtered_with_SAAcut.fits.gz'))


Create the combined data

We will combine the 4C+21.35 source and the albedo photon background, which will be used as our dataset. This only needs to be done once. You can skip this cell if you already have the combined data file.


In [5]:
# Define instance of binned data class:
instance = BinnedData("blazar.yaml")

# Combine files:
input_files = ["/home/srinadb/cosipy/docs/tutorials/spectral_fits/dataset/AlbedoPhotons_3months_unbinned_data_filtered_with_SAAcut.fits.gz","/home/srinadb/cosipy/docs/tutorials/spectral_fits/dataset/4C21p35_flare_3months_unbinned_data_filtered_with_SAAcut.fits.gz"]
instance.combine_unbinned_data(input_files, output_name="4C21p35_flare_AlbedoPhoton_bkg")


Bin the data

You only have to do this once, and after you can start by loading the binned data directly. You can skip this cell if you already have the binned data files.

In [6]:
# Bin 4C+21.35:
flare = BinnedData("blazar.yaml")
flare.get_binned_data(unbinned_data="/home/srinadb/cosipy/docs/tutorials/spectral_fits/dataset/4C21p35_flare_3months_unbinned_data_filtered_with_SAAcut.fits.gz", output_name="4C21p35_flare_binned_data")


In [7]:
# Bin background:
# bg_tot = BinnedData("blazar.yaml")
# bg_tot.get_binned_data(unbinned_data="/home/srinadb/cosipy/docs/tutorials/spectral_fits/dataset/AlbedoPhotons_3months_unbinned_data_filtered_with_SAAcut.fits.gz", output_name="AlbedoPhoton_bkg_binned_data")


In [8]:
# Bin combined data:
data_combined = BinnedData("blazar.yaml")

data_combined.get_binned_data(unbinned_data="/home/srinadb/cosipy/docs/tutorials/spectral_fits/continuum_fit/blazar/4C21p35_flare_AlbedoPhoton_bkg.fits.gz", output_name="4C21p35_flare_AlbedoPhoton_bkg_binned_data")


### Flare

In [9]:
# https://github.com/cositools/cosi-sim/blob/main/cosi_sim/Source_Library/DC3/sources/Extragalactic/4C21p35_flare/4C_21_spectrum_flare.dat

total_integrated_flux = 0.017293201590129727 # ph/cm^2/s
alpha = -2.5

# F_int = (K / (E_piv^alpha * (alpha+1))) * [ E_max^(alpha+1) - E_min^(alpha+1) ]
# K = F_int * (E_piv^alpha * (alpha+1)) / [ E_max^(alpha+1) - E_min^(alpha+1) ]

E_min = 100.0
E_max = 10000.0
E_piv = 1000000

n = total_integrated_flux * (E_piv**alpha) * (1+alpha)
d = (E_max**(1+alpha) - E_min**(1+alpha))

k = n/d
print(k)

2.596576815334794e-14


In [11]:
# https://github.com/cositools/cosi-sim/blob/main/cosi_sim/Source_Library/DC3/sources/Extragalactic/4C21p35_flare/4C_21_spectrum_flare.dat

import math 

# --- Configuration ---
FILENAME = './4C_21_spectrum_flare.dat'  # Name of your data file
ALPHA = -2.5                    # The alpha parameter from the formula
E_PIV_GEV = 1.0                # Pivot energy in GeV

# --- Constants and Conversions ---
E_PIV_KEV = E_PIV_GEV * 1e6    # Convert Pivot Energy to keV
EXPONENT = ALPHA              # The actual exponent in the formula dN/dE = K*(E/E_piv)^(-alpha)

# --- Function to calculate K for a single point ---
def calculate_single_k(energy_kev, dNdE, e_piv_kev, exponent):
    """
    Calculates K for a single energy point using dN/dE = K * (E/E_piv)^exponent.
    Returns K or None if calculation fails.
    """
    if energy_kev <= 0 or e_piv_kev <= 0 or dNdE < 0:
        print(f"Warning: Invalid input for K calculation "
              f"(E={energy_kev:.2e}, dN/dE={dNdE:.2e}). Skipping.")
        return None

    try:
        # Formula: K = dNdE / ( (E / E_piv)^exponent )
        base = energy_kev / e_piv_kev
        denominator = math.pow(base, exponent)

        if denominator == 0:
            print(f"Warning: Denominator is zero for E={energy_kev:.2e}. Skipping K calculation.")
            return None

        K = dNdE / denominator
        return K

    except (OverflowError, ValueError) as e:
        print(f"Error calculating K for E={energy_kev:.2e}: {e}. Skipping.")
        return None
    except ZeroDivisionError:
        print(f"Warning: Division by zero (base likely zero) for E={energy_kev:.2e}. Skipping K calculation.")
        return None


# --- Main Script Logic ---
k_values = []
energies = []
dNdE_values = []

print(f"Reading data from: {FILENAME}")
print(f"Using parameters: alpha={ALPHA}, E_piv={E_PIV_GEV} GeV ({E_PIV_KEV:.1e} keV)")
print(f"Using formula: dN/dE = K * (E / E_piv)^{EXPONENT:.2f}")
print("-" * 30)

# Check if file exists
if not os.path.exists(FILENAME):
    print(f"Error: File not found at '{FILENAME}'")
    exit() # Stop execution if file doesn't exist

# Read the file and process data lines
try:
    with open(FILENAME, 'r') as f:
        for i, line in enumerate(f):
            line = line.strip() # Remove leading/trailing whitespace
            if line.startswith('DP '):
                parts = line.split()
                if len(parts) >= 3:
                    try:
                        energy_kev = float(parts[1])
                        spectrum_dNdE = float(parts[2]) # This is dN/dE

                        # Calculate K for this point
                        k_single = calculate_single_k(energy_kev, spectrum_dNdE, E_PIV_KEV, EXPONENT)

                        # Store values if K calculation was successful
                        if k_single is not None:
                            k_values.append(k_single)
                            energies.append(energy_kev)
                            dNdE_values.append(spectrum_dNdE)
                            # Optional: print K for each point as it's calculated
                            # print(f"  Line {i+1}: E={energy_kev:.3e}, dN/dE={spectrum_dNdE:.3e} -> K={k_single:.3e}")

                    except ValueError:
                        print(f"Warning: Could not parse numbers on line {i+1}: '{line}'. Skipping.")
                    except IndexError:
                         print(f"Warning: Not enough columns on line {i+1}: '{line}'. Skipping.")
                else:
                    print(f"Warning: Malformed 'DP' line {i+1}: '{line}'. Skipping.")

except IOError as e:
    print(f"Error reading file '{FILENAME}': {e}")
    exit()

print("-" * 30)

# --- Calculate and Print Average K ---
if k_values: # Check if the list is not empty
    average_k = sum(k_values) / len(k_values)
    print(f"Successfully calculated K for {len(k_values)} data points.")
    print(f"Average K value: {average_k:.9e} ph cm^-2 s^-1 keV^-1")

    # Optional: Print a summary table
    print("\nIndividual Point Summary:")
    print("Energy (keV) | dN/dE        | Calculated K")
    print("----------------------------------------------")
    for e, dn, k in zip(energies, dNdE_values, k_values):
        print(f"{e:<12.5e} | {dn:<12.5e} | {k:<12.5e}")

else:
    print("No valid K values were calculated from the file.")




Reading data from: ./4C_21_spectrum_flare.dat
Using parameters: alpha=-2.5, E_piv=1.0 GeV (1.0e+06 keV)
Using formula: dN/dE = K * (E / E_piv)^-2.50
------------------------------
------------------------------
Successfully calculated K for 500 data points.
Average K value: 1.049977250e-14 ph cm^-2 s^-1 keV^-1

Individual Point Summary:
Energy (keV) | dN/dE        | Calculated K
----------------------------------------------
1.00000e+02  | 1.05000e-04  | 1.05000e-14 
1.01000e+02  | 1.02600e-04  | 1.05184e-14 
1.02000e+02  | 1.00200e-04  | 1.05285e-14 
1.03000e+02  | 9.79600e-05  | 1.05473e-14 
1.04000e+02  | 9.57300e-05  | 1.05592e-14 
1.05000e+02  | 9.35400e-05  | 1.05675e-14 
1.06000e+02  | 9.14100e-05  | 1.05745e-14 
1.07000e+02  | 8.93300e-05  | 1.05793e-14 
1.08000e+02  | 8.72900e-05  | 1.05809e-14 
1.09000e+02  | 8.53000e-05  | 1.05807e-14 
1.10000e+02  | 8.33500e-05  | 1.05776e-14 
1.11000e+02  | 8.14500e-05  | 1.05730e-14 
1.12000e+02  | 7.95900e-05  | 1.05658e-14 
1.13000e+02 